# Autism Detection

https://github.com/harshita219/Autism_Detection

In [1]:
# Importing the libraries

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from pprint import pprint
from keras.utils import to_categorical

data = ''#add data link over here

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Importing the dataset
dataset = pd.read_csv(data)

# Creating dataset with selected features only
my_dataset = dataset.loc[:,['SUB_ID','SEX','HANDEDNESS_CATEGORY','AGE_AT_SCAN','FIQ','VIQ','PIQ','ADOS_TOTAL',
                            'ADI_R_SOCIAL_TOTAL_A','ADI_R_VERBAL_TOTAL_BV','ADI_RRB_TOTAL_C','ADI_R_ONSET_TOTAL_D',
                            'SUB_IN_SMP','CURRENT_MED_STATUS','DSM_IV_TR']]

## Data preprocessing

In [6]:
# The handedness field contains categorical data, since it has considerably large number of missing values, NaN's are replaced by string 'n/a' (not available) as a new category 
my_dataset.HANDEDNESS_CATEGORY = my_dataset.HANDEDNESS_CATEGORY.fillna('n/a')
        
# Replacing other missing values with mean of respective columns
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
my_dataset.iloc[:, 3:12] = imputer.fit_transform(my_dataset.iloc[:, 3:12])
        
# Dropping all rows which still contain any NaN values
my_dataset = my_dataset.dropna()
my_dataset.head(10)

,SUB_ID,SEX,HANDEDNESS_CATEGORY,AGE_AT_SCAN,FIQ,VIQ,PIQ,ADOS_TOTAL,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,SUB_IN_SMP,CURRENT_MED_STATUS,DSM_IV_TR
0,50002,1,Ambi,16.77,103.0,116.0,89.0,12.000000,16.000000,9.000000,5.000000,4.000000,1,0.0,1.0
1,50003,1,R,24.45,124.0,128.0,115.0,13.000000,27.000000,22.000000,5.000000,3.000000,1,1.0,1.0
2,50004,1,R,19.09,113.0,108.0,117.0,18.000000,19.000000,12.000000,5.000000,3.000000,1,0.0,1.0
3,50005,2,R,13.73,119.0,117.0,118.0,12.000000,23.000000,19.000000,3.000000,4.000000,0,1.0,1.0
4,50006,1,L,13.37,109.0,99.0,119.0,12.000000,13.000000,10.000000,4.000000,3.000000,1,0.0,1.0
5,50007,1,R,17.78,110.0,106.0,112.0,17.000000,21.000000,14.000000,9.000000,1.000000,1,0.0,1.0
6,50008,1,R,32.45,123.0,123.0,114.0,16.000000,24.000000,20.000000,10.000000,2.000000,1,1.0,1.0
7,50009,1,R,33.86,126.0,118.0,128.0,10.000000,20.000000,11.000000,3.000000,2.000000,1,1.0,1.0
8,50010,1,L,35.20,81.0,81.0,93.0,11.062651,19.767196,15.791557,6.084656,3.218855,1,0.0,1.0
9,50011,1,L,16.93,111.0,101.0,120.0,13.000000,24.000000,14.000000,6.000000,1.000000,0,0.0,1.0


In [7]:
print(my_dataset['DSM_IV_TR'].value_counts())

# Features in X and resultant classes in y
X = np.array(my_dataset.iloc[:,1:14].values)
y = np.array(my_dataset.iloc[:,14].values)

0.0    427
1.0    252
2.0     85
3.0     21
Name: DSM_IV_TR, dtype: int64


    The  DSM_IV_TR  field contains the resultant classes for Autism prediction labelled as:

    0 - Control group
    1 - Autism
    2 - Aspergers
    3 - PDD NOS

In [8]:
# One-hot-encoding categorical data of HANDEDNESS_CATEGORY
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = columnTransformer.fit_transform(X)

# Feature Scaling
sc = StandardScaler()
X = sc.fit_transform(X)

# Splitting the dataset into the Training set and Test set in ratio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Model Evaluation and predicting classes

Evaluating seven different models for multiclass classification and recording their accuracies as well as AUC_ROC score for comparing performance. Since ROC uses binary classes, a function is defined which binarizes the labels of predicted and test data.

In [9]:
model_name = ['Random Forest', 'Logistic Regression', 'kNN', 'SVM_rbf', 'ANN', 'XGBoost']
accuracies = [0]*6
auc_roc = [0]*6

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

**Random Forest classifier**:

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The parameters for the random forest training are defined as follows:

*n_estimators*: This is the number of trees in the random forest classification. This model uses 45 trees decided after evaluation of a range of values.
*min_samples_split*: This is the minimum number of samples required to split an internal node. The optimal value for the given  model was found to be 3.
*max_depth*: The maximum depth of the tree, optimal value chosen is 15 here.


The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if *bootstrap=True* (default).

In [10]:
from sklearn.ensemble import RandomForestClassifier
# Using the best parameters combination after tuning hyper-parameters
rf_classifier = RandomForestClassifier(n_estimators = 45, min_samples_split = 3, max_depth = 15, 
                                       bootstrap = False, random_state = 42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracies[0] = accuracy_score(y_test, y_pred)
auc_roc[0] = multiclass_roc_auc_score(y_test, y_pred)

**Logistic Regression classification**:

In the multiclass case, this training algorithm uses the one-vs-rest (OvR) scheme if the ‘multi_class’ option is set to ‘ovr’. *solver* is the algorithm to use in the optimization problem. Maximum number of iterations taken for the solvers to converge are chosen to be 500. It is to be noted that the features (X) are already normalized.

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver = 'saga', multi_class = 'ovr', max_iter = 500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracies[1] = accuracy_score(y_test, y_pred)
auc_roc[1] = multiclass_roc_auc_score(y_test, y_pred)

**K-Nearest Neighbors**:

KNN is a simple supervised classification algorithm we can use to assign a class to new data point. Larger K value leads to smoother decision boundary (less complex model). Smaller K leads to more complex model (may lead to overfitting). Testing accuracy penalizes models that are too complex(over fitting) or not complex enough(underfit). We're using an optimal value of k=5.

In [12]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')
knn.fit(X_train, y_train) 
y_pred = knn.predict(X_test)  

accuracies[2] = accuracy_score(y_test, y_pred)
auc_roc[2] = multiclass_roc_auc_score(y_test, y_pred)

**Support Vector Machine**:

Radial Basis Function (RBF) kernel is applied on the c-support vector classifier. Intuitively, the gamma parameter defines how far the influence of a single training example reaches, with low values meaning ‘far’ and high values meaning ‘close’. The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support vectors.

The C parameter trades off correct classification of training examples against maximization of the decision function’s (One vs All) margin. For larger values of C, a smaller margin will be accepted if the decision function is better at classifying all training points correctly. A lower C will encourage a larger margin, therefore a simpler decision function, at the cost of training accuracy. In other words C behaves as a regularization parameter in the SVM. It is set to the value 0.8 as it gave better scores than default value of 1.

In [13]:
from sklearn.svm import SVC 
svm_model = SVC(kernel = 'rbf', gamma = 'auto', C=0.8)
svm_model.fit(X_train, y_train) 
y_pred = svm_model.predict(X_test) 

accuracies[3] = accuracy_score(y_test, y_pred)
auc_roc[3] = multiclass_roc_auc_score(y_test, y_pred)

**Artificial Neural Network**:

In multi-class classification, the neural network has the same number of output nodes as the number of classes. Each output node belongs to a specific class and outputs a score for that class. Scores from the last layer are passed through a softmax layer. Softmax layer converts the score into probability values. At last, data is classified into a corresponding class, that has the highest probability value. The following ANN has 3 hidden layers and with dropout/regularization rate = 0.1

The classifier is built on 'adam' optimizer and categorical cross-entropy for multiclass. Adam is a replacement optimization algorithm for stochastic gradient descent for training deep learning models. The data is cross validated using k-fold (k=10) on shuffled dataset. The ten accuracies are acquired from cross_val_score and their mean is recorded.

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

def build_classifier():
    classifier = Sequential()
    # Adding the input layer and the first hidden layer 
    classifier.add(Dense(units = 15, activation = 'relu', input_dim = 17))
    # Adding a hidden layer
    classifier.add(Dense(units = 7, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # Adding a hidden layer
    classifier.add(Dense(units = 9, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # Adding a hidden layer
    classifier.add(Dense(units = 7, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # Adding the output layer
    classifier.add(Dense(units = 4, activation = 'softmax'))
    # Compiling the ANN
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

# Encoding the classes in y
y_binary = to_categorical(y)

cv = KFold(n_splits=10, random_state=42, shuffle=True)
classifier = KerasClassifier(build_fn = build_classifier, batch_size=25, epochs=200)
acc_cv = cross_val_score(estimator = classifier, X = X, y = y_binary, cv = cv)

accuracies[4] = acc_cv.mean()
auc_roc[4] = ' '

Epoch 1/200
29/29 [==============================] - 1s 1ms/step - loss: 1.3879 - accuracy: 0.1803
Epoch 2/200
29/29 [==============================] - 0s 1ms/step - loss: 1.3425 - accuracy: 0.4543
Epoch 3/200
29/29 [==============================] - 0s 2ms/step - loss: 1.2913 - accuracy: 0.5485
Epoch 4/200
29/29 [==============================] - 0s 1ms/step - loss: 1.2301 - accuracy: 0.6202
Epoch 5/200
29/29 [==============================] - 0s 1ms/step - loss: 1.1462 - accuracy: 0.6451
Epoch 6/200
29/29 [==============================] - 0s 2ms/step - loss: 1.0356 - accuracy: 0.6729
Epoch 7/200
29/29 [==============================] - 0s 2ms/step - loss: 0.9412 - accuracy: 0.6771
Epoch 8/200
29/29 [==============================] - 0s 1ms/step - loss: 0.8900 - accuracy: 0.6830
Epoch 9/200
29/29 [==============================] - 0s 1ms/step - loss: 0.8560 - accuracy: 0.6967
Epoch 10/200
29/29 [==============================] - 0s 1ms/step - loss: 0.7826 - accuracy: 0.7371
Epoch 11/

**XGBoost Classifier**:

XGBoost (eXtreme Gradient Boosting) is an advanced implementation of gradient boosting algorithm. It's a powerful machine learning algorithm especially where speed and accuracy are concerned. This model requires parameter tuning to improve and fully leverage its advantages over other algorithms. Following parameters were taken into consideration from XGBoost documentation:

    max_depth: Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.
    objective: multi:softmax: set XGBoost to do multiclass classification, also need to set num_class(number of classes)
    subsample: corresponds to the fraction of observations (the rows) to subsample at each step. 
    eta: learning_rate or step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative. 
    

In [15]:
from xgboost import XGBClassifier

# Create a classifier using best params
xgb = XGBClassifier(booster='gbtree', objective='multi:softmax', max_depth = 11, n_estimators = 35,
                    eta = 0.1, subsample  = 0.7, num_class = 4, random_state=42)
# Fit the classifier with the training data
xgb.fit(X_train,y_train)
# Use trained model to predict output of test dataset
y_pred = xgb.predict(X_test)

accuracies[5] = accuracy_score(y_test, y_pred)
auc_roc[5] = multiclass_roc_auc_score(y_test, y_pred)

print('CLASSIFICATION REPORT for XGBoost classifier... \n')
print(classification_report(y_test,y_pred))
print('\nCONFUSION MATRIX for XGBoost classifier... \n')
confusion_matrix(y_test,y_pred)

CLASSIFICATION REPORT for XGBoost classifier... 

              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95        87
         1.0       0.80      0.85      0.82        46
         2.0       0.62      0.40      0.48        20
         3.0       0.00      0.00      0.00         4

    accuracy                           0.85       157
   macro avg       0.58      0.56      0.56       157
weighted avg       0.81      0.85      0.83       157


CONFUSION MATRIX for XGBoost classifier... 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[86,  0,  1,  0],
       [ 4, 39,  3,  0],
       [ 4,  8,  8,  0],
       [ 1,  2,  1,  0]])

### Results 

In summary, the following six machine learning models were developed to classify Autism using 13 selected features from the ABIDE dataset and reducing the subjects to 785. Highest accuracy (84.71 %) and ROC_AUC score (0.74) was achieved by XGBoost algorithm, followed by Random Forest which too was pretty close. The classification report and confusion matrix of the best model is given above.

In [16]:
models = {'Classifiers': model_name, 'Accuracy score': accuracies, 'AUC-ROC Score': auc_roc}
models_df = pd.DataFrame(models, columns = ['Classifiers', 'Accuracy score', 'AUC-ROC Score'] )
models_df

,Classifiers,Accuracy score,AUC-ROC Score
0,Random Forest,0.840764,0.735244
1,Logistic Regression,0.668790,0.599839
2,kNN,0.687898,0.619831
3,SVM_rbf,0.796178,0.694598
4,ANN,0.803749,
5,XGBoost,0.847134,0.747647
